In [1]:
import os
import requests
import csv
from ssgetpy import search, fetch

# Functions provided in Dr. Moore's code
def getMatrixMemoryFootprint(mat):
    return 12 * mat.nnz + 4 * mat.rows

def getILUMemoryFootprint(mat, fill=1):
    return 12 * fill * mat.nnz + 6 * mat.rows

def GetMatrices(RAM, max_fill_factor=50, limit=10**10):
    mats = search(nzbounds=(50000, None), rowbounds=(15000, 20000), colbounds=(15000, 20000), dtype='real', isspd=False, limit=limit)
    matsToKeep = []

    MaxMemory = .8 * (RAM * 1024 ** 3)

    # Square, non-SPD matrices
    for m in mats:
        throwOut = False
        if m.rows != m.cols:
            throwOut = True
        if getMatrixMemoryFootprint(m) + getILUMemoryFootprint(m, fill=max_fill_factor) > MaxMemory:
            throwOut = True

        if not throwOut:
            matsToKeep.append(m)

    return matsToKeep

# Function to download a matrix given its group and name
def download_matrix(group_name, matrix_name, destination_folder):
    url = f"https://suitesparse-collection-website.herokuapp.com/MM/{group_name}/{matrix_name}.tar.gz"
    response = requests.get(url, stream=True)

    if response.status_code == 200:
        # Ensure the destination folder exists
        if not os.path.exists(destination_folder):
            os.makedirs(destination_folder)

        file_path = os.path.join(destination_folder, f"{matrix_name}.tar.gz")
        with open(file_path, "wb") as file:
            for chunk in response.iter_content(chunk_size=1024):
                if chunk:
                    file.write(chunk)
        print(f"Downloaded: {file_path}")
        return file_path
    else:
        print(f"Failed to download {matrix_name}")
        return None

# Function to save matrix details to a CSV file
def save_matrix_details(matrices, csv_file):
    with open(csv_file, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Matrix Name", "Group Name", "Rows", "Columns", "Nonzeros", "URL"])

        for mat in matrices:
            matrix_name = mat.name
            group_name = mat.group
            rows = mat.rows
            cols = mat.cols
            nnz = mat.nnz
            url = f"https://suitesparse-collection-website.herokuapp.com/MM/{group_name}/{matrix_name}.tar.gz"

            writer.writerow([matrix_name, group_name, rows, cols, nnz, url])

# Main function to find and download matrices
def main():
    # Set your desired RAM size in GB and other parameters
    RAM = 16  # Example: 16 GB
    max_fill_factor = 50
    limit = 1000  # Example limit
    destination_folder = "/Users/tonylizza/Documents/Machine_Learning/Thesis_Coding/gflownet-spai/data/large_ILU/more_matrices"  # Folder to save matrices
    csv_file = "matrix_details_1.csv"  # CSV file to save matrix details

    # Find matrices
    matrices = GetMatrices(RAM, max_fill_factor, limit)

    # Save matrix details to CSV file
    save_matrix_details(matrices, csv_file)

    # Download each matrix
    for mat in matrices:
        group_name = mat.group
        matrix_name = mat.name
        download_matrix(group_name, matrix_name, destination_folder)

if __name__ == "__main__":
    main()


Downloaded: /Users/tonylizza/Documents/Machine_Learning/Thesis_Coding/gflownet-spai/data/large_ILU/more_matrices/ex11.tar.gz
Downloaded: /Users/tonylizza/Documents/Machine_Learning/Thesis_Coding/gflownet-spai/data/large_ILU/more_matrices/ex35.tar.gz
Downloaded: /Users/tonylizza/Documents/Machine_Learning/Thesis_Coding/gflownet-spai/data/large_ILU/more_matrices/memplus.tar.gz
Downloaded: /Users/tonylizza/Documents/Machine_Learning/Thesis_Coding/gflownet-spai/data/large_ILU/more_matrices/g7jac060.tar.gz
Downloaded: /Users/tonylizza/Documents/Machine_Learning/Thesis_Coding/gflownet-spai/data/large_ILU/more_matrices/g7jac060sc.tar.gz
Downloaded: /Users/tonylizza/Documents/Machine_Learning/Thesis_Coding/gflownet-spai/data/large_ILU/more_matrices/mark3jac040.tar.gz
Downloaded: /Users/tonylizza/Documents/Machine_Learning/Thesis_Coding/gflownet-spai/data/large_ILU/more_matrices/mark3jac040sc.tar.gz
Downloaded: /Users/tonylizza/Documents/Machine_Learning/Thesis_Coding/gflownet-spai/data/large_I